In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

x0, x1 = 10, 20 # диапазон аргумента функции

test_data_size = 2000 # количество данных для итерации обучения
iterations = 10 # количество итераций обучения
learn_rate = 0.01 # коэффициент переобучения

hiddenSize = 10 # размер скрытого слоя

# функция генерации тестовых величин
def generate_test_values():
    train_x = []
    train_y = []

    for _ in range(test_data_size):
        x = x0+(x1-x0)*np.random.rand()
        y = math.sin(x) # исследуемая функция
        train_x.append([x])
        train_y.append([y])

    return np.array(train_x), np.array(train_y)

# узел на который будем подавать аргументы функции
x = tf.placeholder(tf.float32, [None, 1], name="x")

# узел на который будем подавать значения функции
y = tf.placeholder(tf.float32, [None, 1], name="y")

# скрытый слой
nn = tf.layers.dense(x, hiddenSize,
                     activation=tf.nn.sigmoid,
                     kernel_initializer=tf.initializers.ones(),
                     bias_initializer=tf.initializers.random_uniform(minval=-x1, maxval=-x0),
                     name="hidden")

# выходной слой
model = tf.layers.dense(nn, 1,
                        activation=None,
                        name="output")

# функция подсчёта ошибки
cost = tf.losses.mean_squared_error(y, model)

train = tf.train.GradientDescentOptimizer(learn_rate).minimize(cost)

init = tf.initializers.global_variables()

with tf.Session() as session:
    session.run(init)

    for _ in range(iterations):

        train_dataset, train_values = generate_test_values()

        session.run(train, feed_dict={
            x: train_dataset,
            y: train_values
        })

        if(_ % 1000 == 999):
            print("cost = {}".format(session.run(cost, feed_dict={
                x: train_dataset,
                y: train_values
            })))

    train_dataset, train_values = generate_test_values()

    train_values1 = session.run(model, feed_dict={
        x: train_dataset,
    })

    plt.plot(train_dataset, train_values, "bo",
             train_dataset, train_values1, "ro")
    plt.show()

    with tf.variable_scope("hidden", reuse=True):
        w = tf.get_variable("kernel")
        b = tf.get_variable("bias")
        print("hidden:")
        print("kernel=", w.eval())
        print("bias = ", b.eval())

    with tf.variable_scope("output", reuse=True):
        w = tf.get_variable("kernel")
        b = tf.get_variable("bias")
        print("output:")
        print("kernel=", w.eval())
        print("bias = ", b.eval())

/tmp/ipykernel_5457/984724474.py:33: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  nn = tf.layers.dense(x, hiddenSize,


ValueError: Variable hidden/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/home/siarhei/.local/lib/python3.8/site-packages/keras/engine/base_layer_v1.py", line 421, in add_weight
    variable = self._add_variable_with_custom_getter(
  File "/home/siarhei/.local/lib/python3.8/site-packages/keras/legacy_tf_layers/base.py", line 456, in add_weight
    variable = super(Layer, self).add_weight(
  File "/home/siarhei/.local/lib/python3.8/site-packages/keras/layers/core/dense.py", line 145, in build
    self.kernel = self.add_weight(
  File "/home/siarhei/.local/lib/python3.8/site-packages/keras/engine/base_layer_v1.py", line 2033, in _maybe_build
    self.build(input_shapes)
  File "/home/siarhei/.local/lib/python3.8/site-packages/keras/engine/base_layer_v1.py", line 743, in __call__
    self._maybe_build(inputs)
